In [1]:
import os
from dotenv import load_dotenv
from account import Account, load_accounts, load_mock_accounts
from action import load_actions, store_actions
from executor import MockExecutor, Executor, exec_next_action

load_dotenv()

CUSTOMER_API_KEY = os.getenv("CUSTOMER_API_KEY")
CUSTOMER_API_KEY_SECRET = os.getenv("CUSTOMER_API_KEY_SECRET")
CLIENT_ID = os.getenv("CLIENT_ID")
CLIENT_SECRET = os.getenv("CLIENT_SECRET")

MOCK = True

soc_account, dev_account = load_mock_accounts() if MOCK else load_accounts()
print(soc_account.name, dev_account.name)

{'token_type': 'bearer', 'expires_in': 7200, 'access_token': 'LUlJMTFGM1JoN1FHSkEyTFJBZmFrUy1lNWw3QWJiUE5SaEhLMUJ0X0ZoNk5TOjE2ODM0NTMzNjU0NjY6MToxOmF0OjE', 'scope': ['follows.read', 'offline.access', 'tweet.write', 'like.write', 'users.read', 'tweet.read'], 'refresh_token': 'WUdpcWxuajhrRk9lZmlTWGRMbW82b2dVS2ZhZnJsMUVCQ2N1aksxNndicFRNOjE2ODM0NTMzNjU0NjY6MTowOnJ0OjE', 'expires_at': 1683460642.943203}
{'token_type': 'bearer', 'expires_in': 7200, 'access_token': 'M3lHT1dObGVqZFhWcTQ2YlFRNnZsclphOUg2TkpyV0JBY0czbXhhcFNUaExMOjE2ODM0NTMzNjYyNTI6MTowOmF0OjE', 'scope': ['follows.read', 'offline.access', 'tweet.write', 'like.write', 'users.read', 'tweet.read'], 'refresh_token': 'TERoTHZMVkpVNHJqdHo1MG9KUkRzTl9GNlN0UVNrOW1nMHd6b0FQdUFBN1k2OjE2ODM0NTMzNjYyNTI6MToxOnJ0OjE', 'expires_at': 1683460643.733951}
sonsofcryptolabs web3dev


In [5]:
actions = load_actions(soc_account, dev_account, MOCK)
executor = Executor()
actions = exec_next_action(executor, actions)
store_actions(actions, MOCK)

Posting sonsofcryptolabs 0 "Privacy is not a crime."

- Anonymous ./content/cpa/000.png
Account sonsofcryptolabs LUlJMTFGM1JoN1FHSkEyTFJBZmFrUy1lNWw3QWJiUE5SaEhLMUJ0X0ZoNk5TOjE2ODM0NTMzNjU0NjY6MToxOmF0OjE


TooManyRequests: 429 Too Many Requests
Too Many Requests

In [15]:
import pandas
import tweepy
import pathlib
from post import Post
from typing import List

def image_path(id: str, path: str) -> str:
    paths = os.listdir(path)
    if f"{id}.png" in paths:
        return f"{path}/{id}.png"
    if f"{id}.jpg" in paths:
        return f"{path}/{id}.jpg"
    if f"{id}.jpeg" in paths:
        return f"{path}/{id}.jpeg"
    return None

def post_from(id: str, text: str, path: str) -> Post:
    return Post(
        int(curr_id),
        text=curr_text if len(curr_text) > 0 else None,
        img_path=image_path(curr_id, path)
    )

PATH = pathlib.Path(__file__).parent.resolve()
LIST_CONTENT_PATH = f"{PATH}/content"

name = "mem"
path = LIST_CONTENT_PATH + f"/{name}"

posts: List[Post] = list()
curr_text: str = ""
curr_id: str = 0
with open(path + "/text.txt", "r") as file:
    while True:
        line = file.readline()
        if not line:
            break
        if line[0:3] == "#ID":
            curr_id = line[3:].strip()
            curr_text = curr_text.strip()
            if len(curr_text) <= 280:
                posts.append(post_from(curr_id, curr_text, path))
            else:
                print("dropping", curr_id, len(curr_text))
            curr_text = ""
        else:

            curr_text += line

for post in posts:
    print(post.id, post.img_path)



0 None
1 ./content/mem/001.jpg
2 ./content/mem/002.png
3 ./content/mem/003.jpeg
4 ./content/mem/004.jpg
5 ./content/mem/005.jpg
6 ./content/mem/006.jpg
7 ./content/mem/007.jpg
8 ./content/mem/008.jpeg
9 ./content/mem/009.png
10 ./content/mem/010.png
11 ./content/mem/011.png
12 ./content/mem/012.jpg
13 ./content/mem/013.png
14 ./content/mem/014.jpg
15 ./content/mem/015.jpg
16 ./content/mem/016.jpg
17 ./content/mem/017.jpg
18 ./content/mem/018.jpg
19 ./content/mem/019.png
20 ./content/mem/020.png
21 ./content/mem/021.jpg
22 ./content/mem/022.png
23 ./content/mem/023.png
24 ./content/mem/024.png
25 ./content/mem/025.png
26 ./content/mem/026.png
27 ./content/mem/027.png
28 ./content/mem/028.png
29 ./content/mem/029.png


In [23]:
# telegram bot
import telegram
import os
from dotenv import load_dotenv
import asyncio

load_dotenv()


TG_BOT_TOKEN = os.getenv("TG_BOT_TOKEN")

async def some_fn():
    print(TG_BOT_TOKEN)
    bot = telegram.Bot(TG_BOT_TOKEN)
    async with bot:
        bot.send_message(text='Testing TG bot', chat_id="-1001481837102")

def workaround():
    bot = telegram.Bot(TG_BOT_TOKEN)
    # bot.send_message(text='Testing TG bot', chat_id="-1001481837102")
    asyncio.ensure_future(bot.send_message(text='Testing TG bot', chat_id="-1001481837102"))

workaround()

